In [45]:
import pandas as pd
import sqlite3

In [46]:
#Configuração do pandas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Bronze

In [47]:
# Leitura de arquivo CSV
# Conversão de csvpara parquet 
ac = pd.read_csv('./medallion/bronze/acidentes2025.csv', sep=';', index_col="id" ,encoding='latin-1')
dt = pd.read_csv('./medallion/bronze/datatran2025.csv', sep=';', index_col="id" ,encoding='latin-1') 

ac.to_csv('./medallion/silver/acidentes2025.csv')
dt.to_csv('./medallion/silver/datatran2025.csv')

# Silver

In [48]:
#Leitura dos arquivos Parquet
ac_silver = pd.read_csv('./medallion/silver/acidentes2025.csv')
dt_silver = pd.read_csv('./medallion/silver/datatran2025.csv')

In [49]:
#Novo data frame
df = ac_silver.merge(dt_silver, how='inner')

Tratamentos

In [50]:
df.loc[df['pesid'] == 0 , 'pesid'] = -1
df.loc[df['ano_fabricacao_veiculo'] == 0 , 'ano_fabricacao_veiculo'] = -1
df.loc[df['id_veiculo'] == 0, 'id_veiculo'] = -1
df.loc[df['br'] == 0 , 'br'] = -1
df.loc[df['km'] == 0 , 'km'] = -1
df.loc[df['idade'] == 0 , 'idade'] = -1
df['km'] = (df['km'].str.replace(',', '.', regex=False).pipe(pd.to_numeric, errors='coerce'))

In [51]:
df.fillna('Não informado', inplace=True)

In [52]:
#Latitude e longitude como float
df['latitude'] = (df['latitude']
    .astype(str)
    .str.replace(',', '.', regex=False)
    .astype(float)
)

df['longitude'] = (
    df['longitude']
    .astype(str)
    .str.replace(',', '.', regex=False)
    .astype(float)
)

In [53]:
contagem_nulos = df.isnull().sum()

colunas_com_nulos = contagem_nulos[contagem_nulos > 0].to_frame(name='Contagem de Valores Nulos')
print("Colunas com valores nulos:")
print(colunas_com_nulos)

Colunas com valores nulos:
Empty DataFrame
Columns: [Contagem de Valores Nulos]
Index: []


# Gold

In [54]:
# --- Arquivo final limpo e formatado ---

df.to_csv('./medallion/gold/acidentes_datatran2025.csv', index=False)
conn = sqlite3.connect('./medallion/gold/acidentes.db')

df.to_sql(
    name='acidentes',
    con=conn,
    if_exists='replace',
    index=False
)

conn.close()